In [46]:
import tensorflow as tf
import numpy as py
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=Warning)
from sklearn import preprocessing
import urllib.request
import os

dataset_url='https://biostat.app.vumc.org/wiki/pub/Main/DataSets/titanic3.csv'
dataset_folder='./datasets/'
dataset_path='datasets/titanic3.csv'

if not os.path.exists(dataset_folder):
    os.makedirs(dataset_folder)
if not os.path.isfile(dataset_path):
    result=urllib.request.urlretrieve(dataset_url,dataset_path)
    print('download:',result)
else: print(dataset_path,'already exists')

datasets/titanic3.csv already exists


In [47]:
df=pd.read_csv(dataset_path)
selected_cols=['survived','name','pclass','sex','age','sibsp','parch','fare','embarked']
df_selected=df[selected_cols]

'''
df_selected.isnull().sum()
df_selected[df_selected.isnull().values==True]
'''

'\ndf_selected.isnull().sum()\ndf_selected[df_selected.isnull().values==True]\n'

In [48]:
df_selected['age']=df_selected['age'].fillna(df_selected['age'].mean())
df_selected['fare']=df_selected['fare'].fillna(df_selected['fare'].mean())
df_selected['embarked']=df_selected['embarked'].fillna('S')
df_selected['embarked']=df_selected['embarked'].map({'C':0,'Q':1,'S':2}).astype(int)
df_selected['sex']=df_selected['sex'].map({'female':0,'male':1}).astype(int)
df_selected=df_selected.drop(['name'],axis=1)

In [49]:
feature=df_selected.values[:,1:]
label=df_selected.values[:,0]
featured=preprocessing.MinMaxScaler(feature_range=(0,1)).fit_transform(feature)